Learn to add two 2-digit numbers.
90..99+n is never seen during training but gets an accuracy in the 60%
Memorized 0..9+n has really bad accuracy of 6%, perhaps because of single digit versus all other double digit numbers
All other memorized ranges have an accuracy around 94%


In [1]:
from gptbench import Sample, LogFlag, Train, empty_config

Create train and validation dataset:
python prepare_addition.py ../data/add2.txt 2 --sep="\n" --split=0.9

Creates add2.train.txt and add2.val.txt

In [2]:
with open('../data/add2.val.txt', 'r', newline=None) as f:
    val_data = f.read()
val_data[:500]

'90+0=90\n90+1=91\n90+2=92\n90+3=93\n90+4=94\n90+5=95\n90+6=96\n90+7=97\n90+8=98\n90+9=99\n90+10=100\n90+11=101\n90+12=102\n90+13=103\n90+14=104\n90+15=105\n90+16=106\n90+17=107\n90+18=108\n90+19=109\n90+20=110\n90+21=111\n90+22=112\n90+23=113\n90+24=114\n90+25=115\n90+26=116\n90+27=117\n90+28=118\n90+29=119\n90+30=120\n90+31=121\n90+32=122\n90+33=123\n90+34=124\n90+35=125\n90+36=126\n90+37=127\n90+38=128\n90+39=129\n90+40=130\n90+41=131\n90+42=132\n90+43=133\n90+44=134\n90+45=135\n90+46=136\n90+47=137\n90+48=138\n90+49=139\n90+50=140\n90+51=141\n'

In [3]:
do = Train('add2', log_mask=LogFlag.ALL)

# set datasets
do.set_datasets('char', train_path='../data/add2.train.txt', val_path='../data/add2.val.txt')

# set config settings
cfg = empty_config()
cfg.train.log_period=-0.5
cfg.model.set(n_layer=6, n_head=6, n_embd=90, block_size=16)
cfg.trainer.set(batch_size=128)

# and init a new model with config
if do.can_resume():
    do.init_resume(cfg)
else:
    do.init_new(cfg)
    do.train(iter_count=10000)    

New random seed 2012991848
Loading checkpoint from ./checkpoints\add2
Checkpoint: iter_num=9800, eval loss=0.8883152008056641 2
Dataset train_path: ../data/add2.train.txt, val_path_or_train_split: ../data/add2.val.txt, vocab_size: 13


In [4]:
do.val_dataset.get_src_data()[:500]

'90+0=90\n90+1=91\n90+2=92\n90+3=93\n90+4=94\n90+5=95\n90+6=96\n90+7=97\n90+8=98\n90+9=99\n90+10=100\n90+11=101\n90+12=102\n90+13=103\n90+14=104\n90+15=105\n90+16=106\n90+17=107\n90+18=108\n90+19=109\n90+20=110\n90+21=111\n90+22=112\n90+23=113\n90+24=114\n90+25=115\n90+26=116\n90+27=117\n90+28=118\n90+29=119\n90+30=120\n90+31=121\n90+32=122\n90+33=123\n90+34=124\n90+35=125\n90+36=126\n90+37=127\n90+38=128\n90+39=129\n90+40=130\n90+41=131\n90+42=132\n90+43=133\n90+44=134\n90+45=135\n90+46=136\n90+47=137\n90+48=138\n90+49=139\n90+50=140\n90+51=141\n'

In [5]:
str(do.get_config())

'seed=0 \nsample: max_len=100 count=1 start_text=56+45= start_after== stop_before=\n emit_start=1 start_text_sep=| flush=1 eot_stop=0 top=0 temp=1.0 multiline_prompt=0 \ntrain: sample_num=1254400 eval_loss=0.8883152008056641 train_loss=0.5735212564468384 val_loss=0.8883152008056641 log_period=-0.5 eval_period=100 eval_type=2 eval_iters=100 eval_save_checkpt=1 sample_period=1000 batch_end_callback=default \ndataset: class_name=char train_path=../data/add2.train.txt val_path_or_train_split=../data/add2.val.txt \nmodel: device=auto dtype=float32 model_type=None n_layer=6 n_head=6 n_embd=90 vocab_size=13 block_size=16 dropout=0.1 \ntrainer: n_workers=4 batch_size=128 max_samples=None grad_norm_clip=1.0 opti=1 learning_rate=0.0001 adamw_betas=(0.9, 0.95) adamw_weight_decay=0.1 '

In [56]:
do.sample(start_text="3+80=")

143

In [ ]:
def test_accuracy(first_number, second_number):
    qa = []
    for a in range(100):
        for b in range(100):
            qa.append((f'{a}+{b}=', f'{a+b}'))

    qa=qa[first_number*100:second_number*100]
    
    sep = do.get_config().sample.start_text_sep
    start_text = sep.join([q for q,a in qa])

    ans = []
    do.sample(start_text, dest=ans)

    corr=0
    for i in range(len(qa)):
        q,a = qa[i]
        #print(q,a,ans[i])
        if a == ans[i]:
            corr+=1
        
    print(f'{corr/len(qa):.3f}')


In [72]:
test_accuracy(90, 100)

0.677


In [71]:
test_accuracy(0, 10)

0.069


In [76]:
test_accuracy(10, 90)

0.936
